In [ ]:
#Imports of all necessary Python Packages
import sklearn as skl
import numpy as np
import pandas as pd
from time import time

#Classifiers
from sklearn.svm import SVC
from sklearn.svm import NuSVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.decomposition import PCA

from sklearn.dummy import DummyClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier


#Evaluation

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

In [ ]:
#Here import the data to classify
data = np.random.rand(100,5)
labels = np.random.randint(2,size=100)

In [ ]:
# Feature Extraction
pca = PCA(n_components = 2)
data_decomposed = pca.fit_transform(data)

In [ ]:
#SVM
classfSVM = SVC(C=1, kernel='rbf', gamma='auto', tol=1e-3)

In [ ]:
#SVM with control of the %(nu) of support vectors 
classfNuSVM = NuSVC(nu=0.5, kernel='rbf', gamma='auto', tol=1e-3)

In [ ]:
#LDA
classfLDA = LinearDiscriminantAnalysis()

In [ ]:
#QDA
classfQDA = QuadraticDiscriminantAnalysis()

In [ ]:
#KNN
classfKNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

In [ ]:
#DecisionTree
classfDecisionTree = DecisionTreeClassifier()

In [ ]:
#AdaBoost
classfAdaBoost = AdaBoostClassifier(base_estimator=None, learning_rate=1, n_estimators=50)

In [ ]:
#RandomForest
classfRandForest = RandomForestClassifier(n_estimators=10, n_jobs=-1)

In [ ]:
#Test classifier on all data or preferably on a Test set:
#DON'T trust these results, there is an high chance of overfitting
#Use ONLY to get a feel of the time Cross-Validation will take

classf = classfSVM
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.20, random_state=42)

t0 = time()
classf.fit(train_data,train_labels)
print "Score of Classifier: " + str(classf.score(test_data,test_labels))
print("done in %0.3fs." % (time() - t0))

In [ ]:
# Analyze Confusion Matrix
true_labels = labels
test_data = data
classf = classfSVM
predicted_labels = classf.predict(test_data)
confusion_matrix(true_labels, predicted_labels)

In [ ]:
# Select parameters to use in Cross-Validation
classf_cv = classfSVM
data_cv = data_decomposed
N_CV = 10

In [ ]:
# Cross Validation
t0 = time()
scores = cross_validation.cross_val_score(classf_cv,data_cv,labels, n_jobs=-1, cv = N_CV)
print "Scores: "
for i,score in enumerate(scores):
    print '\t' + str(i) + ':\t' + str(score) 
print "\nCrossval Final score: "+ str(sum(scores)/N_CV)
print("\nCross val done in %0.3fs." % (time() - t0))